In [191]:
#Next steps - 
#1 - find a way to get all the results from all the pages
#2 - continue running through SVM student example with this dataset 
##  - https://gt.bootcampcontent.com/GT-Coding-Boot-Camp/gt-atl-data-pt-06-2020-u-c/blob/master/DataViz-Content/21-Machine-Learning/2/Activities/08-Stu_SVM/Solved/Stu_SVM.ipynb
#3 - identify the test accuracy. If it's below 60% look up ways to tweak the model 
# - spend no more than 1 hour teaking the model 
#4 - time to visualize. A few questions to consider: 
#Qa - what trends exist with movies that have actually scored 7.5 avg rating
#Qb - what % of future movies are expected to have a 7.5+ rating 
#Qc - how does the % of future movies with a 7.5+ rating compare to the past? 
#Qd - what are key features of 7.5 internatinal movies compared to 7.5 US movies? 
#Conclusion - as a movie executive, these are the features most likily to score well with viewers
#Bonus - what's was revenue of recent releases with the top 5 highest average vote scores?

In [209]:
import pandas as pd
import requests
import json
from pprint import pprint
import numpy as np
from pandas.io.json import json_normalize

In [227]:
a = pd.read_csv("a.csv")
a.columns
df = a[['id','title','popularity','vote_count','vote_average','original_language','release_date']]
df

,id,title,popularity,vote_count,vote_average,original_language,release_date
0,531219,Roald Dahl's The Witches,1497.123,698,6.9,en,2020-10-26
1,718444,Rogue,562.083,354,5.8,en,2020-08-20
2,594718,Sputnik,475.982,50,6.4,ru,2020-09-10
3,531499,The Tax Collector,444.615,179,6.0,en,2020-08-07
4,425001,The War with Grandpa,347.616,121,6.0,en,2020-08-27
...,...,...,...,...,...,...,...
195,741032,Danyka,6.292,0,0.0,en,2020-11-26
196,653756,The Mole Agent,4.199,0,0.0,es,2020-12-11
197,660330,Night Shift,5.035,34,6.3,fr,2020-05-21
198,745589,Villa caprice,4.251,0,0.0,fr,2020-11-25


In [118]:
#clearn your data so that all your features are categorical or numerical
#feature a: popularity is fine as is
#feature b: release data needs to be in categorical form by month 
#feature c: original language is fine as is 

In [213]:
#make release_data categorical. Parse by year, month, day then drop year
# new_releases_df['release_month'] = 
# df['year'] = pd.DatetimeIndex(df['release_date']).year
# df['r_month'] = pd.DatetimeIndex(df['release_date']).month
# df['r_day'] = pd.DatetimeIndex(df['release_date']).day
# df2 = df[['id','title','popularity','vote_count','vote_average','origional_language','release_date','r_year','r_month','r_day']]
# final_data_clean2 .head(3)

In [ ]:
#create a csv for movies that haven't been released

In [233]:
movies_to_predict = df[(df['vote_count'] == 0)]
movies_to_predict
train_model_movies = df[(df['vote_count'] > 0)]
train_model_movies
# movies_to_predict.to_csv("a_movies_to_predict.csv")
train_model_movies.to_csv("a_train_model_movies.csv")

In [ ]:
# MODELING TIME
#use logistical regression to determine if a new movie will get an above average mean vote (ie. a 5.0+ vote average)
#why logisticall regression???? because we want a simple y/n prediction 

In [234]:
#Step 01 - import
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns

#Step 02 - Look at the shape of the dataframe
train_model_movies.shape

(98, 7)

In [41]:
#input variables are: popularity, vote_count, vote_average
#ouput variable: is this true? "yes" - vote_average is > 5.0 for future movies 

In [235]:
#Create two bins for vote_average: above and at/below average
#first let's look at the distribution of vote_averages
train_model_movies['vote_average'].unique()

array([6.9, 5.8, 6.4, 6. , 6.6, 5.5, 6.3, 7.1, 6.7, 5.7, 6.8, 7.9, 6.5,
       7.5, 7.8, 4.4, 4.3, 6.1, 5.9, 7.2, 7. , 6.2, 4.7, 8.5, 8.1, 4.9,
       8. , 7.6, 1. , 5.1, 8.7, 7.7, 4.5, 3.8, 5.6, 7.4, 5.4])

In [240]:
#next let's group into either "above_avg" (5.0+) or "not" (>=5.0)
train_model_movies['avg_vote_5+'] = np.where((train_model_movies['vote_average']>=5), 1, 0)
train_model_movies.tail(3)

<ipython-input-240-d7e6788081ac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_model_movies['avg_vote_5+'] = np.where((train_model_movies['vote_average']>=5), 1, 0)


,id,title,popularity,vote_count,vote_average,original_language,release_date,avg_vote_7.5+,avg_vote_5+
193,613319,Selfie,4.927,48,5.6,fr,2020-01-15,0,1
197,660330,Night Shift,5.035,34,6.3,fr,2020-05-21,0,1
199,573650,Bangla,6.157,74,6.1,it,2019-05-16,0,1


In [243]:
train_model_movies['avg_vote_5+'].value_counts()

1    90
0     8
Name: avg_vote_5+, dtype: int64

In [244]:
#Step 03 - Define the features of the target
X = train_model_movies[['popularity','vote_count','vote_average']]
y = train_model_movies[['avg_vote_5+']]


In [245]:
#Split the dataset in to test and train using sklearn before building the SVM 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

/Users/jennieleegruber/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVC(kernel='linear')

In [246]:
y_predict = svclassifier.predict(X_test)

In [247]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict))

[[ 3  0]
 [ 0 17]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00        17

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



In [248]:
from sklearn.svm import SVC
svclassifier2 = SVC(kernel='rbf')
svclassifier2.fit(X_train, y_train)

/Users/jennieleegruber/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVC()

In [249]:
y_pred2 = svclassifier2.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred2))
print(classification_report(y_test, y_pred2))

[[ 0  3]
 [ 0 17]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.85      1.00      0.92        17

    accuracy                           0.85        20
   macro avg       0.42      0.50      0.46        20
weighted avg       0.72      0.85      0.78        20



/Users/jennieleegruber/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [103]:
#Source: https://intellipaat.com/blog/tutorial/machine-learning-tutorial/svm-algorithm-in-python/

In [260]:
# ### trying this another way !!!!!
# #define the factor you're looking to predict
# target = train_model_movies['avg_vote_5+']
# #define the unique options of the predicted outcome
# target_names = ['1','0']
# #define the dataset without the factor you're trying to predict
# data = train_model_movies.drop("avg_vote_5+", axis=1)
# data2 = data.drop("avg_vote_7.5+", axis=1)

In [262]:
# data2.head(3)

In [144]:
#SVM Modeling - New Attempt

array([0, 1])